### Import Libraries

In [1]:
import pandas as pd
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.chunk import RegexpParser
from nltk import sent_tokenize, word_tokenize, pos_tag
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
string.punctuation

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

### Preprocessing

In [2]:
# read data (data yang digunakan adalah data sample 500 baris agar pemrosesannya tidak lama)

data = pd.read_csv('data/patent.csv')
data = data[["title", "abstract_localized.text"]]
data.head()

,title,abstract_localized.text
0,Fire detection system based on artificial inte...,The present disclosure relates to a fire detec...
1,NaN,The present disclosure relates to a fire detec...
2,Embedding artificial intelligence for balancin...,Responsive to a CPU load of a specific access ...
3,NaN,Responsive to a CPU load of a specific access ...
4,Optimization techniques for artificial intelli...,"Methods, apparatuses and computer readable med..."


In [3]:
# add . in the end of title
def add_period(text):
    if pd.notna(text):
        return text + ". "
    else:
        return text

# Apply the function to the 'title' column
data['title'] = data['title'].apply(add_period)

# merge title and abstract column
data['title_and_abstract'] = data['title'].fillna('') + data['abstract_localized.text']

# Print the updated DataFrame
data.head()

,title,abstract_localized.text,title_and_abstract
0,Fire detection system based on artificial inte...,The present disclosure relates to a fire detec...,Fire detection system based on artificial inte...
1,NaN,The present disclosure relates to a fire detec...,The present disclosure relates to a fire detec...
2,Embedding artificial intelligence for balancin...,Responsive to a CPU load of a specific access ...,Embedding artificial intelligence for balancin...
3,NaN,Responsive to a CPU load of a specific access ...,Responsive to a CPU load of a specific access ...
4,Optimization techniques for artificial intelli...,"Methods, apparatuses and computer readable med...",Optimization techniques for artificial intelli...


In [4]:
# check missing value

print(data.isna().sum())

title                      234
abstract_localized.text      0
title_and_abstract           0
dtype: int64


In [5]:
# # remove missing value (tidak perlu karena kolom title and abstract tidak memiliki missing value)

# data.dropna(inplace=True)
# data.head()

In [6]:
# lowering text
data['title_and_abstract']= data['title_and_abstract'].apply(lambda x: x.lower())

# Print the updated DataFrame
data['title_and_abstract'].head()

0    fire detection system based on artificial inte...
1    the present disclosure relates to a fire detec...
2    embedding artificial intelligence for balancin...
3    responsive to a cpu load of a specific access ...
4    optimization techniques for artificial intelli...
Name: title_and_abstract, dtype: object

In [7]:
# Function to tokenize each sentence, tokenize each word from sentence, then remove stopwords
def tokenize_and_remove_punctuation(text):
    # Tokenize each sentence
    sentences = sent_tokenize(text)
    
    # Remove punctuation and tokenize each word in each sentence
    tokenized_sentences = []
    for sentence in sentences:
        tokens = word_tokenize(sentence)
        # Remove punctuation
        tokens = [token for token in tokens if token not in string.punctuation]
        tokenized_sentences.append(tokens)
    
    return tokenized_sentences

# Tokenize each sentence and remove punctuation
data['tokenized'] = data['title_and_abstract'].apply(tokenize_and_remove_punctuation)

# Print the updated DataFrame
print(data['tokenized'][0])

[['fire', 'detection', 'system', 'based', 'on', 'artificial', 'intelligence', 'and', 'fire', 'detection', 'method', 'based', 'on', 'artificial', 'intelligence'], ['the', 'present', 'disclosure', 'relates', 'to', 'a', 'fire', 'detection', 'system', 'based', 'on', 'ai', 'and', 'a', 'fire', 'detection', 'method', 'based', 'on', 'ai', 'which', 'detects', 'fire', 'by', 'using', 'deep', 'learning'], ['the', 'fire', 'detection', 'system', 'includes', 'a', 'camera', 'part', 'generating', 'video', 'images', 'an', 'object', 'extraction', 'part', 'extracting', 'a', 'motion', 'object', 'from', 'the', 'video', 'images', 'a', 'video', 'image', 'conversion', 'part', 'generating', 'a', 'first', 'background', 'removal', 'image', 'an', 'image', 'division', 'part', 'dividing', 'the', 'first', 'background', 'removal', 'image', 'with', 'a', 'plurality', 'of', 'division', 'lines', 'and', 'an', 'analyzing', 'part', 'generating', 'an', 'abnormal', 'signal', 'and', 'a', 'normal', 'signal']]


### POS Tag

In [8]:
# Function to perform POS tagging on each token
def pos_tag_tokens(tokenized_sentences):
    pos_tagged_sentences = []
    for sentence_tokens in tokenized_sentences:
        pos_tags = pos_tag(sentence_tokens)
        pos_tagged_sentences.append(pos_tags)
    return pos_tagged_sentences

# Perform POS tagging on tokenized sentences
data['pos_tagged'] = data['tokenized'].apply(pos_tag_tokens)

# Print the updated DataFrame
print(data['pos_tagged'][0])

[[('fire', 'NN'), ('detection', 'NN'), ('system', 'NN'), ('based', 'VBN'), ('on', 'IN'), ('artificial', 'JJ'), ('intelligence', 'NN'), ('and', 'CC'), ('fire', 'NN'), ('detection', 'NN'), ('method', 'NN'), ('based', 'VBN'), ('on', 'IN'), ('artificial', 'JJ'), ('intelligence', 'NN')], [('the', 'DT'), ('present', 'JJ'), ('disclosure', 'NN'), ('relates', 'VBZ'), ('to', 'TO'), ('a', 'DT'), ('fire', 'NN'), ('detection', 'NN'), ('system', 'NN'), ('based', 'VBN'), ('on', 'IN'), ('ai', 'NN'), ('and', 'CC'), ('a', 'DT'), ('fire', 'NN'), ('detection', 'NN'), ('method', 'NN'), ('based', 'VBN'), ('on', 'IN'), ('ai', 'NN'), ('which', 'WDT'), ('detects', 'VBZ'), ('fire', 'NN'), ('by', 'IN'), ('using', 'VBG'), ('deep', 'JJ'), ('learning', 'NN')], [('the', 'DT'), ('fire', 'NN'), ('detection', 'NN'), ('system', 'NN'), ('includes', 'VBZ'), ('a', 'DT'), ('camera', 'NN'), ('part', 'NN'), ('generating', 'VBG'), ('video', 'NN'), ('images', 'NNS'), ('an', 'DT'), ('object', 'JJ'), ('extraction', 'NN'), ('part'

In [9]:
# Function to lemmatize tokens
def lemmatize_tokens(pos_tagged_sentences):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentences = []
    for sentence_tags in pos_tagged_sentences:
        lemmatized_tokens = [(lemmatizer.lemmatize(token, pos=get_wordnet_pos(pos_tag)), pos_tag) for token, pos_tag in sentence_tags]
        lemmatized_sentences.append(lemmatized_tokens)
    return lemmatized_sentences

# Function to map POS tags to WordNet POS tags
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif treebank_tag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return nltk.corpus.wordnet.ADV
    else:
        return nltk.corpus.wordnet.NOUN  # Default to noun if POS tag not recognized

# Lemmatize the tokens in 'pos_tagged_sentences' column
data['lemmatized'] = data['pos_tagged'].apply(lemmatize_tokens)

# Print the updated DataFrame
print(data['lemmatized'][0])

[[('fire', 'NN'), ('detection', 'NN'), ('system', 'NN'), ('base', 'VBN'), ('on', 'IN'), ('artificial', 'JJ'), ('intelligence', 'NN'), ('and', 'CC'), ('fire', 'NN'), ('detection', 'NN'), ('method', 'NN'), ('base', 'VBN'), ('on', 'IN'), ('artificial', 'JJ'), ('intelligence', 'NN')], [('the', 'DT'), ('present', 'JJ'), ('disclosure', 'NN'), ('relate', 'VBZ'), ('to', 'TO'), ('a', 'DT'), ('fire', 'NN'), ('detection', 'NN'), ('system', 'NN'), ('base', 'VBN'), ('on', 'IN'), ('ai', 'NN'), ('and', 'CC'), ('a', 'DT'), ('fire', 'NN'), ('detection', 'NN'), ('method', 'NN'), ('base', 'VBN'), ('on', 'IN'), ('ai', 'NN'), ('which', 'WDT'), ('detect', 'VBZ'), ('fire', 'NN'), ('by', 'IN'), ('use', 'VBG'), ('deep', 'JJ'), ('learning', 'NN')], [('the', 'DT'), ('fire', 'NN'), ('detection', 'NN'), ('system', 'NN'), ('include', 'VBZ'), ('a', 'DT'), ('camera', 'NN'), ('part', 'NN'), ('generate', 'VBG'), ('video', 'NN'), ('image', 'NNS'), ('an', 'DT'), ('object', 'JJ'), ('extraction', 'NN'), ('part', 'NN'), ('e

### POS Chunk

In [10]:
# pos chunk verb noun pairs

def extract_verb_noun_pairs(tagged_texts):
    chunks = []
    for tagged_sentence in tagged_texts:
        result = filter_verb_noun_pairs(tagged_sentence)
        chunks.append(result)
    chunks = [item for sublist in chunks for item in sublist]
    return chunks

def filter_verb_noun_pairs(tagged_sentence):
    verb_noun_pairs = []
    for i in range(len(tagged_sentence) - 1):
        word, tag = tagged_sentence[i]
        next_word, next_tag = tagged_sentence[i + 1]
        if tag.startswith('VB') and next_tag.startswith('NN'):
            verb = word
            noun = next_word
            verb_noun_pairs.append((verb, noun))
    return verb_noun_pairs

# Apply chunking and extraction to the 'tagged_sentences' column
data['chunked_verb_noun'] = data['lemmatized'].apply(extract_verb_noun_pairs)

# Print the resulting DataFrame with chunked data
data['chunked_verb_noun']

0      [(detect, fire), (generate, video), (analyze, ...
1      [(detect, fire), (generate, video), (analyze, ...
2      [(balance, access), (process, load), (enable, ...
3                [(threshold, value), (determine, rssi)]
4      [(receive, annotation), (determine, document),...
                             ...                        
495                [(learn, model), (further, comprise)]
496    [(optimize, hyperparameter), (optimize, hyperp...
497    [(optimize, hyperparameter), (ai, model), (eva...
498    [(provide, system), (provide, ai), (compute, e...
499    [(provide, system), (provide, ai), (compute, e...
Name: chunked_verb_noun, Length: 500, dtype: object

In [11]:
# POS chunk new pattern (Pak Indra)

# define pattern
pattern = r'''Chunk: {(<ADJ|VB\w*>+<\.|RP|IN|CC|PRP\w*>*<DT>*<NN\w*|RB\w*>+)+}'''
chunk_parser = RegexpParser(pattern)

def extract(tagged_texts):
    chunks = []
    for tagged_text in tagged_texts:
        tree = chunk_parser.parse(tagged_text)
        tree = extract_chunks(tree)
        tree = trees_to_tuples(tree)
        chunks.append(tree)
    chunks = [item for sublist in chunks for item in sublist]
    return chunks

def extract_chunks(tree):
    chunks = []
    
    if isinstance(tree, nltk.Tree):
        if tree.label() != 'S':  # Exclude sentence-level chunks if any
            chunks.append(tree)
        for subtree in tree:
            chunks.extend(extract_chunks(subtree))
    
    return chunks

def trees_to_tuples(tree_list):
    tuple_list = [tuple(leaf[0] for leaf in tree.leaves()) for tree in tree_list]
    return tuple_list

# Apply chunking and extraction to the 'tagged_sentences' column
data['chunked_new_pattern'] = data['lemmatized'].apply(extract)

# Print the resulting DataFrame with chunked data
data['chunked_new_pattern']

0      [(base, on, ai), (base, on, ai), (detect, fire...
1      [(base, on, ai), (base, on, ai), (detect, fire...
2      [(balance, access, point, process, load), (ena...
3      [(threshold, value), (determine, rssi, value),...
4      [(comprise, select, from, a, pool), (be, annot...
                             ...                        
495    [(comprise, receive, from, a, client, device),...
496    [(optimize, hyperparameter, tuples), (optimize...
497    [(optimize, hyperparameter, tuples), (ai, mode...
498    [(be, provide, system), (provide, ai, system),...
499    [(be, provide, system), (provide, ai, system),...
Name: chunked_new_pattern, Length: 500, dtype: object

### Export Result

In [12]:
# hasile

patent_task = data[["title_and_abstract", "chunked_verb_noun", "chunked_new_pattern"]]
patent_task.to_excel('export_result/chunk1.xlsx', index=False)